# Trajectory estimation

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import os

%matplotlib inline
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

## Read and plot dataset

In [ ]:
dimension = 2 # can be 2 or 3.
n_measurements = -1 # set to None or -1 to use all. 
chosen_distance = 'distance_tango_2D'#'distance_median_0'

sigma = 10 # noise for simulated measurements

out_dir = 'experiments/robot_test/'
anchorsfile = 'experiments/anchors.csv'

name = 'circle2_double'
df = pd.read_pickle(out_dir + name + '_0_calibrated.pkl')
df.reset_index(inplace=True, drop=True)
df.timestamp -= df.timestamp.values[0]
df.head()

In [ ]:
tango_df = df[df.system_id=='Tango']
plt.figure()
sns.scatterplot(x='px', y='py', hue='timestamp', data=tango_df)
plt.title('2D of Tango data')
plt.xlabel('x [m]'); plt.ylabel('y [m]')
plt.figure()
plt.plot(tango_df.timestamp, tango_df.pz)
plt.title('z of Tango data')
plt.xlabel('time [s]'); plt.ylabel('z [m]')

## Construct D, basis and anchors

In [ ]:
from evaluate_dataset import read_anchors_df
anchors_df = read_anchors_df(anchorsfile)

anchor_names = sorted(df[df.system_id=='RTT'].anchor_name.unique().tolist())
n_anchors = len(anchor_names)

# if we do not have all we could do something elegant with pandas. 
anchors = anchors_df.loc[anchors_df.anchor_name.isin(anchor_names), ['px', 'py', 'pz']].values.T

for anchor_name in anchor_names:
    anchor_coord = anchors_df.loc[anchors_df.anchor_name==anchor_name, ['px','py']].values
    points = tango_df.loc[:, ['px', 'py']].values
    vecs = points.reshape((-1, 2)) - anchor_coord.reshape((1, 2))
    distances = np.linalg.norm(vecs.astype(np.float32), axis=1)

    df_rtt0 = df[df.anchor_name == anchor_name]

    plt.figure()
    plt.scatter(tango_df.timestamp, distances, label='tango', s=2)
    for distance in ['distance_tango_2D', 'distance_tango', 'distance', 'distance_median_all', 'distance_median_0']:
        plt.scatter(df_rtt0.timestamp, df_rtt0.loc[:, distance], s=2, label=distance)
    plt.xlabel('time [s]'); plt.ylabel('distance [m]'); plt.legend()

In [ ]:
from trajectory_creator import get_trajectory
#from trajectory import Trajectory
from global_variables import TAU, ROBOT_HEIGHT

traj = get_trajectory(name + '.csv')
# scramble coefficients.
traj.set_coeffs(seed=3)

times = df.iloc[:n_measurements].timestamp.unique().astype(np.float32)
n_times = len(times)

#lengths = tango_df.length.values[indices]
#distances = np.cumsum(lengths)
speed = 1/6. # 10m/min
distances = speed * times * TAU # TODO need to understand why TAU.

# get times in terms of trajectory
times_corr, _, _  = traj.get_times_from_distances(arbitrary_distances=distances, plot=False)
basis = traj.get_basis(times=times_corr)

fig, ax = plt.subplots()
ax = traj.plot(basis=basis, alpha=0.5)

## real measurements

In [ ]:
D_topright_real = np.zeros((n_times, n_anchors))

for i, t in enumerate(times):
    this_slice = df[(df.system_id=='RTT') & (df.timestamp == t)]
    
    # this can be done more elegantly with pandas
    for anchor_name in this_slice.anchor_name:
        a_id = anchor_names.index(anchor_name)
        distance = this_slice.loc[this_slice.anchor_name==anchor_name, chosen_distance].values[0]
        
        if dimension == 3:
            D_topright_real[i, a_id] = distance ** 2
        else:
            if chosen_distance != 'distance_tango_2D':
                anchor_height = anchors_df[anchors_df.anchor_name==anchor_name].pz
                distance_sq = distance ** 2 - (anchor_height - ROBOT_HEIGHT) ** 2
            else:
                distance_sq = distance ** 2
            D_topright_real[i, a_id] = distance_sq
        
plt.figure()
plt.matshow(D_topright[:40, :].T)
plt.title('first 40 rows of D_topright, transposed')

## simulated measurements

In [ ]:
# now we need the real trajectory, so we do not scramble.
traj = get_trajectory(name + '.csv')

points = traj.get_sampling_points(basis=basis)
plt.figure()
plt.scatter(*points[:2])
plt.axis('equal')

from measurements import get_D_topright, add_noise
D_topright_sim = get_D_topright(anchors=anchors[:dimension], samples=points[:2])
np.random.seed(1)
D_topright_sim = add_noise(D_topright_sim, sigma, noise_to_square=False)

## Estimate trajectory

In [ ]:
from solvers import alternativePseudoInverse

#D_topright = D_topright_sim
D_topright = D_topright_real
print(anchors.shape)

coeffs = alternativePseudoInverse(D_topright, anchors[:2], basis, 
                                  weighted=False)
print(coeffs)
#print(traj.coeffs)
trajectory_estimated = traj.copy()
trajectory_estimated.set_coeffs(coeffs=coeffs)
plt.figure()
ax = trajectory_estimated.plot()
points = trajectory_estimated.get_sampling_points(basis=basis)
points_true = traj.get_sampling_points(basis=basis)
ax.scatter(*points[:2])
ax.scatter(*points_true[:2])
for a_coord, a_name in zip(anchors[:2].T, anchor_names):
    ax.scatter(*a_coord, color='green'); ax.annotate(s=a_name, xy=a_coord, color='green')
plt.axis('equal')
if dimension > 2:
    plt.figure()
    plt.plot(times, points[2])
    plt.plot(times, points_true[2])

When setting sigma to 10 in simulated measurements, then the recovered trajectory looks very similar to the reconstruction using real noisy measurements. We should investigate why. Hypothesis: we take samples at the same time so maybe we oversample in one region compared to the other region.  